In [1]:
import pandas as pd
from docx import Document
import os

In [2]:
file_path = 'iphone_data.csv'
data = pd.read_csv(file_path)

In [3]:
data_cleaned = data.drop([0, 1]).reset_index(drop=True)

In [4]:
question_texts = data.iloc[0, :].to_dict()

pid_column = 'Q26'
date_column = 'RecordedDate' 

columns_of_interest = [
    "Q15", "Q10", "Q24", 
    "Q3_4", "Q16", "Q3_5", "Q18", 
    "Q3_6", "Q19", "Q3_7", "Q20", 
    "Q3_8", "Q21", "Q3_9", "Q22"
]

In [5]:
columns_of_interest_with_texts = [question_texts[col] for col in columns_of_interest]

In [6]:
data_filtered = data_cleaned[[pid_column, date_column] + columns_of_interest]

In [7]:
data_filtered[pid_column] = data_filtered[pid_column].str.strip().str.lower()

/var/folders/vr/fynfqqj539zdh2hh2gwbpjn40000gp/T/ipykernel_31139/331908638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered[pid_column] = data_filtered[pid_column].str.strip().str.lower()


In [8]:
grouped_data = data_filtered.groupby(pid_column).apply(lambda df: df.to_dict('records'))

In [9]:
output_folder = 'cleaned_iphone'
os.makedirs(output_folder, exist_ok=True)

for pid, entries in grouped_data.items():
    try:
        doc = Document()
        doc.add_heading(f'Participant ID: {pid}', 0)
        
        for entry in entries:
            entry_date = entry.get(date_column, 'Unknown Date')
            doc.add_heading(f'[Entry: {entry_date}]', level=1)
            for col, question_text in zip(columns_of_interest, columns_of_interest_with_texts):
                response = entry.get(col)
                if response:
                    doc.add_paragraph(f"[{col}] {question_text}: \n- {response}")
            doc.add_paragraph('\n') 
        
        file_name = os.path.join(output_folder, f'{pid}.docx')
        doc.save(file_name)
        
    except Exception as e:
        print(f"An error occurred while processing PID: {pid}. Error: {e}")

print("Export completed.")

An error occurred while processing PID: https://ischooluw.co1.qualtrics.com/jfe/form/sv_5ii6sxmtgfd8wlu. Error: [Errno 2] No such file or directory: 'cleaned_iphone/https://ischooluw.co1.qualtrics.com/jfe/form/sv_5ii6sxmtgfd8wlu.docx'
Export completed.
